In [28]:
import pandas as pd
from scipy.stats import levene, bartlett
from statsmodels.formula.api import ols
import statsmodels.api as sm
import pingouin as pg
from scipy import stats

In [6]:
data_anxity = pd.read_csv("anxity.csv")
data_anxity

,Unnamed: 0,id,group,observation,reads
0,1,1,grp1,t1,14.1
1,2,1,grp1,t2,14.4
2,3,1,grp1,t3,14.1
3,4,2,grp1,t1,14.5
4,5,2,grp1,t2,14.6
...,...,...,...,...,...
130,131,44,grp3,t2,16.4
131,132,44,grp3,t3,15.1
132,133,45,grp3,t1,19.0
133,134,45,grp3,t2,17.2


In [8]:
anxity_subset = data_anxity[["group","observation","reads"]]

In [14]:
#Normality check for distribution
statistic, p_value = stats.shapiro(data_anxity["reads"])
print("statistics:", statistic)
print("p_value:", p_value)

statistics: 0.9869789036748995
p_value: 0.23155222959112665


In [16]:
#Extracttt unique levels of observation
observation_levels = data_anxity["observation"].unique()
observation_levels


array(['t1', 't2', 't3'], dtype=object)

In [ ]:
from scipy.stats import levene
grouped_reads = [data_anxity[data_anxity["observation"] == level]["reads"] for level in ["t1", "t2", "t3"]]
grouped_reads

In [22]:
#perform levene test
statistic, p_value =levene(*grouped_reads)
print("levene's test statistcs:", statistic)
print("p_value:", p_value)

levene's test statistcs: 2.3410322270615245
p_value: 0.10021171478874775


In [25]:
mod = ols("reads ~ observation", data= anxity_subset).fit()

In [30]:
aov_table = sm.stats.anova_lm(mod, type=2)
print(aov_table)

                df      sum_sq    mean_sq          F    PR(>F)
observation    2.0   66.578815  33.289407  10.839924  0.000044
Residual     132.0  405.372000   3.071000        NaN       NaN


In [31]:
#pairwise tukeyhst test
tukey_result = sm.stats.multicomp.pairwise_tukeyhsd(data_anxity["reads"],data_anxity["observation"])

In [32]:
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    t1     t2    -0.78 0.0914 -1.6557 0.0957  False
    t1     t3  -1.7178    0.0 -2.5935 -0.842   True
    t2     t3  -0.9378 0.0327 -1.8135 -0.062   True
---------------------------------------------------


In [33]:
import pingouin as pg

In [35]:
aov = pg.anova(data_anxity, dv = "reads", between= "observation", detailed=True)
print(aov)

        Source          SS   DF         MS          F     p-unc       np2
0  observation   66.578815    2  33.289407  10.839924  0.000044  0.141072
1       Within  405.372000  132   3.071000        NaN       NaN       NaN


In [36]:
pt = pg.pairwise_tukey(dv = "reads", between= "observation" ,data = data_anxity)

In [37]:
print(pt)

    A   B    mean(A)    mean(B)      diff        se         T   p-tukey  \
0  t1  t2  16.915556  16.135556  0.780000  0.369444  2.111281  0.091421   
1  t1  t3  16.915556  15.197778  1.717778  0.369444  4.649629  0.000024   
2  t2  t3  16.135556  15.197778  0.937778  0.369444  2.538349  0.032734   

    hedges  
0  0.47325  
1  0.97601  
2  0.49713  
